<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_1_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Introducción

La aceleración es la tasa de variación de la velocidad de un objeto cuando se mueve. Si el objeto en cuestión mantiene su velocidad de forma constante, entonces no está acelerando, algo que sí hará cuando su velocidad cambia. La aceleración se mide en metros por segundo al cuadrado, y lo hace en base al tiempo que le lleva pasar de una velocidad a otra, o bien en una fuerza que se aplica sobre el objeto. [1]

Se puede calcular la aceleración de un cuerpo mediante la siguiente fórmula:

## <center>$\frac{vf-vi}{tf-ti} = {a}$

A continuación, calcularemos la aceleración de varios cuerpos utilizando GPU.


#2 Armado del ambiente
Instalar en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 23.2MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 81kB 13.6MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=0ac5de53df9ed009bbff660540b1269a4c1ffaef8b67543833d35ac168949b03
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=9c72a071e38e0e189a6dcaf17c80828c2452c4f74c844a1c46ae1ebb20ab2769
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

In [ ]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

cantidadVehiculos =   1000#@param {type: "integer", min:0}
tiempoSegundos =   30 #@param {type:"slider", min:0, max:100, step:1}

# --------------------------------------------

from datetime import datetime
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy

#Defino variables
acelTotal_cpu = 0
vMax = 120 # [m/s]

tiempo_total = datetime.now()

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

try: 

  if cantidadVehiculos < 0:
    raise ValueError("La cantidad de vehículos debe ser mayor a 0");

  if tiempoSegundos == 0:
    raise ValueError("El tiempo debe ser mayor a cero");

  # --------------------------------------------
  # CPU - Defino la memoria de los vectores en cpu.
  vi_cpu = numpy.random.randint(vMax, size=cantidadVehiculos)
  vi_cpu = vi_cpu.astype( numpy.float32() )

  vf_cpu = numpy.random.randint(vMax, size=cantidadVehiculos)
  vf_cpu = vf_cpu.astype( numpy.float32() )
  acel_cpu = numpy.empty_like( vf_cpu )

  # CPU - reservo la memoria GPU.
  vf_gpu = cuda.mem_alloc( vf_cpu.nbytes )
  vi_gpu = cuda.mem_alloc( vi_cpu.nbytes )

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod( vf_gpu, vf_cpu )
  cuda.memcpy_htod( vi_gpu, vi_cpu )

  # CPU - Defino la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_acel( int n, float time, float *vf, float *vi)
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if( idx < n )
    {
      vf[idx] = (vf[idx] - vi[idx])/time;
    }
  }
  """) 

  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel_acel")

  tiempo_gpu = datetime.now()

  # GPU - Ejecuta el kernel.
  dim_hilo = 256
  dim_bloque = numpy.int( (cantidadVehiculos+dim_hilo-1) / dim_hilo )

  kernel(numpy.int32(cantidadVehiculos),numpy.float32(tiempoSegundos), vf_gpu, vi_gpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )
  tiempo_gpu = datetime.now() - tiempo_gpu

  # GPU - Copio el resultado desde la memoria GPU.
  cuda.memcpy_dtoh( acel_cpu, vf_gpu )

  acelPromedio = numpy.mean(acel_cpu) 
  tiempo_total = datetime.now() - tiempo_total

  print( "Cantidad de elementos: ", cantidadVehiculos )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
  print("Tiempo GPU: ", tiempo_en_ms( tiempo_gpu   ), "[ms]" )
  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Aceleración promedio: ", acelPromedio, "[m/s2]" )

except ValueError:
  print (valerr)
except Exception as e:
  print("Error:",e.args)
                       

Cantidad de elementos:  1000
Thread x:  256 , Bloque x: 4
Tiempo GPU:  0.086 [ms]
Tiempo Total:  2.31 [ms]
Aceleración promedio:  -0.0602 [m/s2]


#4 Tabla de pasos

 Procesador | Función | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño de vectores desde Colab.
CPU      |  @param                | Lectura del tiempo.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( Cantidad_N ) | Inicializa los vectoes vf, vi y acel.
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  SourceModule()        | Define el código del kernel 
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_tx/dim_bx         | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU.
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria vf a CPU memoria acel.
CPU      | numpy.mean(vf_cpu)     | Realiza el cálculo del promedio, guardando el valor en acelPromedio 
CPU      |  print()               | Informo los resultados.

#5 Conclusiones

Realicé la ejecución del cuaderno diez veces para el cálculo de una aceleración promedio para 1000 vehículos en 30 segundos. Obteniendo como resultado los siguientes valores:

Tiempo GPU:  0.079 [ms] | Tiempo Total:  2.133 [ms] <br>
Tiempo GPU:  0.102 [ms] | Tiempo Total:  2.370 [ms] <br>
Tiempo GPU:  0.095 [ms] | Tiempo Total:  2.274 [ms] <br>
Tiempo GPU:  0.128 [ms] | Tiempo Total:  3.654 [ms] <br>
Tiempo GPU:  0.093 [ms] | Tiempo Total:  2.002 [ms] <br>
Tiempo GPU:  0.074 [ms] | Tiempo Total:  1.895 [ms] <br>
Tiempo GPU:  0.090 [ms] | Tiempo Total:  2.006 [ms] <br>
Tiempo GPU:  0.098 [ms] | Tiempo Total:  2.435 [ms] <br>
Tiempo GPU:  0.064 [ms] | Tiempo Total:  1.457 [ms] <br>
Tiempo GPU:  0.086 [ms] | Tiempo Total:  2.310 [ms] <br>

Podemos observar, que el tiempo de ejecución promedio del programa **[2.253 ms]** es mucho menor al obtenido sin utilizar GPU **[3.983 ms]**, esto se debe a que el cálculo de la aceleración de los cuerpos se calcula en simultáneo.

El punto más relevante en este ejercicio es la utilización de la función Kernel, la cuál nos permite ejecutar el algoritmo en una dimensión utilizando hilos.

En cuanto a lecciones aprendidas, comprendí el funcionamiento de hilos. <br>
Como mejora, se podrían agregar gráficos ilustrativos de los resultados obtenidos.

#6 Bibliografía

[1] Aceleracion: [Referencia](https://www.fisicalab.com/apartado/aceleracion)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Numpy: [Referencia](https://numpy.org/doc/1.16/reference/routines.random.html)

[4] Sliders Parametros: [Referencia](https://colab.research.google.com/notebooks/forms.ipynb)

[5] Documentación PyCUDA: [WEB](https://documen.tician.de/pycuda/index.html)

[6] Repositorio de PyCUDA: [WEB](https://pypi.python.org/pypi/pycuda)